## 1. Initializations and Dataset Download

In [ ]:
# !pip install opendatasets --quiet
# import opendatasets as od
# od.download("https://www.kaggle.com/datasets/mssmartypants/rice-type-classification")

## 2. Imports

Lets start by getting all our imports, keep in mind that PyTorch is not automatically detects and trains on GPU, you have to tell it to use cuda. In case you want to train on Mac Silicon replace cuda with mps.

## 3. Dataset

## 4. Data Preprocessing

## 5. Data Splitting

## 6. Dataset Object

## 7. Training Hyper Parameters

## 8. Data Loaders

## 9. Model Class

## 10. Model Creation

## 11. Loss and Optimizer

## 12. Training

## 13. Testing

## 14. Plotting and Visualizations (Optional)

## 15. Inference